In [ ]:
# -*- coding: utf-8 -*-
"""
DIC-9345 - Projet 2: Traduction Automatique Neuronale (TAN) - EN->RU
Version utilisant NLLB-600M pour viser un meilleur score.
Entraînement sur opus_books complet, 5 époques.
Utilise do_eval=False. Batch size 4.
Correction: Modification de la stratégie de sauvegarde pour économiser l'espace disque.
"""

# @title 1. Installation des bibliothèques nécessaires
# Installe les bibliothèques Hugging Face (transformers, datasets), SacreBLEU et Accelerate.
# Sentencepiece est nécessaire pour le tokenizer NLLB.
!pip install transformers[torch] datasets sacrebleu accelerate evaluate sentencepiece -q

print("Installation terminée.")

# @title 2. Importations et Configuration Initiale
import os
# Silence XLA/TensorFlow CUDA warnings
os.environ["TF_CPP_MIN_LOG_LEVEL"] = "2"
# Désactive Weights & Biases (si non utilisé)
os.environ["WANDB_DISABLED"] = "true"

import torch
import numpy as np
# Utilisation de 'evaluate' au lieu de 'load_metric' pour les métriques
from datasets import load_dataset
import evaluate # Nouvelle façon de charger les métriques
from transformers import (
    AutoTokenizer, # Utilisation de AutoTokenizer pour NLLB
    AutoModelForSeq2SeqLM,
    DataCollatorForSeq2Seq,
    Seq2SeqTrainingArguments,
    Seq2SeqTrainer
)

# Configuration pour Anglais -> Russe
# CHANGEMENT: Utilisation du modèle NLLB-600M
MODEL_CHECKPOINT = "facebook/nllb-200-distilled-600M"
# Codes langues pour NLLB (peuvent différer des codes Helsinki)
# Voir: https://huggingface.co/facebook/nllb-200-distilled-600M#languages-covered
SOURCE_LANG_NLLB = "eng_Latn" # Code NLLB pour Anglais
TARGET_LANG_NLLB = "rus_Cyrl" # Code NLLB pour Russe

# Noms des colonnes dans le dataset opus_books
SOURCE_LANG_DATA = "en"
TARGET_LANG_DATA = "ru"

DATASET_NAME = "opus_books"
DATASET_CONFIG = "en-ru"

# Limites pour l'exemple (COMMENTÉES POUR UTILISER TOUTES LES DONNÉES)
# MAX_TRAIN_SAMPLES = 10000
# MAX_VAL_SAMPLES = 1000
# MAX_TEST_SAMPLES = 1000
MAX_INPUT_LENGTH = 128 # Peut nécessiter ajustement pour NLLB si les phrases sont longues
MAX_TARGET_LENGTH = 128

# Vérification explicite du device GPU
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Utilisation du périphérique : {device}")
if torch.cuda.is_available():
    print(f"Nom du GPU: {torch.cuda.get_device_name(0)}")
else:
    print("Aucun GPU détecté, utilisation du CPU.")


# @title 3. Chargement et Prétraitement des Données

# Charger le jeu de données (opus_books en-ru)
try:
    raw_datasets_full = load_dataset(DATASET_NAME, DATASET_CONFIG)
    print(f"Dataset {DATASET_NAME} ({DATASET_CONFIG}) chargé.")
    print(raw_datasets_full)
    # Division du split 'train'
    train_test_split = raw_datasets_full['train'].train_test_split(test_size=0.05, seed=42)
    train_val_split = train_test_split['train'].train_test_split(test_size=0.05, seed=42)

    raw_datasets = {
        'train': train_val_split['train'],
        'validation': train_val_split['test'],
        'test': train_test_split['test']
    }
    print("Dataset divisé en train/validation/test.")
    print(f"Tailles réelles des splits - Train: {len(raw_datasets['train'])}, Validation: {len(raw_datasets['validation'])}, Test: {len(raw_datasets['test'])}")

except Exception as e:
    print(f"Erreur lors du chargement ou de la division du dataset {DATASET_NAME} ({DATASET_CONFIG}): {e}")
    raise e

# Charger le tokenizer NLLB
# Utilisation de AutoTokenizer, spécification des langues source/cible
tokenizer = AutoTokenizer.from_pretrained(
    MODEL_CHECKPOINT,
    src_lang=SOURCE_LANG_NLLB,
    tgt_lang=TARGET_LANG_NLLB
)
print(f"Tokenizer chargé pour {MODEL_CHECKPOINT} (src={SOURCE_LANG_NLLB}, tgt={TARGET_LANG_NLLB})")

# Fonction de prétraitement adaptée pour NLLB
def preprocess_function(examples):
    # Utiliser les clés du dataset ('en', 'ru')
    inputs = [ex[SOURCE_LANG_DATA] for ex in examples["translation"]]
    targets = [ex[TARGET_LANG_DATA] for ex in examples["translation"]]

    # Tokenisation des entrées (Anglais)
    model_inputs = tokenizer(inputs, max_length=MAX_INPUT_LENGTH, truncation=True)

    # Tokenisation des sorties (Russe) comme labels
    labels = tokenizer(text_target=targets, max_length=MAX_TARGET_LENGTH, truncation=True)

    model_inputs["labels"] = labels["input_ids"]
    return model_inputs

# Appliquer le prétraitement aux datasets
num_cpus = os.cpu_count()
print(f"Utilisation de {num_cpus} coeurs pour le prétraitement.")
tokenized_datasets = {}
for split, dataset in raw_datasets.items():
     cols_to_remove = ['id', 'translation']
     tokenized_datasets[split] = dataset.map(
         preprocess_function,
         batched=True,
         remove_columns=cols_to_remove,
         num_proc=num_cpus,
         desc=f"Tokenizing {split} split..."
     )

print("Prétraitement terminé.")
print("Structure après tokenisation (exemple train):", tokenized_datasets["train"])


# Utiliser les datasets complets
train_dataset = tokenized_datasets["train"]
eval_dataset = tokenized_datasets["validation"]
test_dataset_tokenized = tokenized_datasets["test"]
test_dataset_raw = raw_datasets["test"]

print(f"Taille du jeu d'entraînement utilisé: {len(train_dataset)} exemples")
print(f"Taille du jeu d'évaluation utilisé (référence): {len(eval_dataset)} exemples")
print(f"Taille du jeu de test utilisé: {len(test_dataset_tokenized)} exemples (tokenisé)")
print(f"Taille du jeu de test utilisé: {len(test_dataset_raw)} exemples (raw)")


# @title 4. Chargement du Modèle et Configuration de l'Entraînement

# Charger le modèle NLLB
model = AutoModelForSeq2SeqLM.from_pretrained(MODEL_CHECKPOINT)
model.to(device)
print(f"Modèle {MODEL_CHECKPOINT} chargé et envoyé sur {device}.")

# Nom du run pour le suivi
run_name = f"nllb-600m-finetuned-{SOURCE_LANG_DATA}-to-{TARGET_LANG_DATA}-opus-full-5e-v2" # Run NLLB v2

# Arguments d'entraînement
training_args = Seq2SeqTrainingArguments(
    output_dir=run_name,
    do_train=True,
    do_eval=False,              # PAS d'évaluation pendant l'entraînement
    logging_strategy="steps",
    logging_steps=100,
    save_strategy="no",         # CORRECTION: Ne pas sauvegarder pendant l'entraînement
    learning_rate=5e-6,
    per_device_train_batch_size=4,
    per_device_eval_batch_size=4,
    gradient_accumulation_steps=4,
    weight_decay=0.01,
    # save_total_limit=2,       # Non pertinent si save_strategy="no"
    num_train_epochs=5,
    predict_with_generate=True,
    fp16=torch.cuda.is_available(),
    push_to_hub=False,
    generation_max_length=MAX_TARGET_LENGTH,
    # report_to="wandb",
)

# Data Collator (standard)
data_collator = DataCollatorForSeq2Seq(tokenizer, model=model)

# Métriques d'évaluation
sacrebleu_metric = evaluate.load("sacrebleu")
chrf_metric = evaluate.load("chrf")

def postprocess_text(preds, labels):
    preds = [pred.strip() for pred in preds]
    labels = [[label.strip()] for label in labels]
    return preds, labels

# compute_metrics (inchangé, mais sera appelé par predict)
def compute_metrics(eval_preds):
    preds, labels = eval_preds
    if isinstance(preds, tuple):
        preds = preds[0]

    # Ignorer les tokens de padding (-100) avant décodage
    preds = np.where(preds != -100, preds, tokenizer.pad_token_id)
    labels = np.where(labels != -100, labels, tokenizer.pad_token_id)

    decoded_preds = tokenizer.batch_decode(preds, skip_special_tokens=True)
    decoded_labels = tokenizer.batch_decode(labels, skip_special_tokens=True)

    # Nettoyage simple
    decoded_preds, decoded_labels = postprocess_text(decoded_preds, decoded_labels)

    try:
        bleu_result = sacrebleu_metric.compute(predictions=decoded_preds, references=decoded_labels)
        # Calcul chrF standard
        chrf_result = chrf_metric.compute(predictions=decoded_preds, references=decoded_labels)
        result = {"bleu": bleu_result["score"], "chrf": chrf_result["score"]}
    except Exception as e:
        print(f"Erreur lors du calcul des métriques: {e}")
        print("Prédictions:", decoded_preds[:2]) # Afficher les 2 premières pour débogage
        print("Labels:", decoded_labels[:2])
        result = {"bleu": 0.0, "chrf": 0.0} # Retourner 0 en cas d'erreur

    # Calcul longueur moyenne (optionnel)
    try:
        prediction_lens = [np.count_nonzero(pred != tokenizer.pad_token_id) for pred in preds]
        result["gen_len"] = np.mean(prediction_lens)
    except:
        result["gen_len"] = 0 # Gérer le cas où preds est vide ou autre erreur

    result = {k: round(v, 4) for k, v in result.items()}
    return result

# Initialiser le Trainer
trainer = Seq2SeqTrainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    tokenizer=tokenizer,
    data_collator=data_collator,
    compute_metrics=compute_metrics,
)

print(f"Configuration de l'entraînement terminée ({run_name}, prêt pour GPU, do_eval=False).")


# @title 5. Lancement de l'Entraînement (Fine-tuning)
# ATTENTION: Ceci prendra BEAUCOUP plus de temps (plusieurs heures / toute la nuit).
print(f"Début de l'entraînement ({run_name}) sur GPU...")
try:
    train_result = trainer.train()
    # Sauvegarde du modèle final uniquement APRES la fin de l'entraînement
    print("Entraînement terminé. Sauvegarde du modèle final...")
    trainer.save_model()
    print("Modèle final sauvegardé.")
    # Log et sauvegarde des métriques d'entraînement
    metrics = train_result.metrics
    metrics["train_samples"] = len(train_dataset)
    trainer.log_metrics("train", metrics)
    trainer.save_metrics("train", metrics)
    trainer.save_state() # Sauvegarde l'état du trainer (utile pour reprise éventuelle)

except Exception as e:
    print(f"Une erreur est survenue pendant l'entraînement : {e}")
    if "CUDA out of memory" in str(e):
        print("Erreur 'CUDA out of memory'. Essayez de réduire 'per_device_train_batch_size' ou 'gradient_accumulation_steps'.")
    # Afficher l'erreur spécifique si c'est celle vue précédemment
    if "enforce fail at inline_container.cc" in str(e):
        print("Erreur interne PyTorch/XLA détectée pendant l'entraînement.")
    print("L'entraînement a été interrompu.")
    # Essayer de sauvegarder le dernier état même si erreur
    try:
        print("Tentative de sauvegarde du dernier état du modèle...")
        trainer.save_model(os.path.join(run_name, "checkpoint-interrupted"))
        trainer.save_state(os.path.join(run_name, "checkpoint-interrupted"))
        print("Dernier état sauvegardé dans 'checkpoint-interrupted'.")
    except Exception as save_e:
        print(f"Impossible de sauvegarder le dernier état après interruption: {save_e}")


# @title 6. Évaluation sur le Jeu de Test
# S'assurer que le modèle est chargé (soit le dernier après entraînement complet, soit depuis un checkpoint si interrompu)
# Si l'entraînement a été interrompu, il faudrait manuellement charger le checkpoint sauvegardé avant predict.
# Pour l'instant, on suppose que l'entraînement s'est terminé ou que le modèle dans 'trainer' est utilisable.

print("Début de l'évaluation finale sur le jeu de test avec le modèle actuel...")
model.eval()

try:
    # Utiliser trainer.predict() pour l'évaluation finale sur le jeu de test
    predict_results = trainer.predict(test_dataset_tokenized, metric_key_prefix="test")

    metrics = predict_results.metrics
    metrics["test_samples"] = len(test_dataset_raw)

    print(f"----- Résultats de l'évaluation finale sur le jeu de test ({run_name}) -----")
    print(f"Score SacreBLEU: {metrics.get('test_bleu', 'N/A'):.4f}")
    print(f"Score chrF: {metrics.get('test_chrf', 'N/A'):.4f}")

    # Recalculons pour avoir tous les détails de SacreBLEU si nécessaire
    if predict_results.predictions is not None:
        preds = predict_results.predictions
        preds = np.where(preds != -100, preds, tokenizer.pad_token_id)
        decoded_preds = tokenizer.batch_decode(preds, skip_special_tokens=True)
        # Références du dataset brut
        references = [ex['translation'][TARGET_LANG_DATA] for ex in test_dataset_raw]
        cleaned_preds, cleaned_labels = postprocess_text(decoded_preds, references)

        try:
            final_bleu_metric = evaluate.load("sacrebleu")
            test_bleu_results_detailed = final_bleu_metric.compute(predictions=cleaned_preds, references=cleaned_labels)

            print(f"(Recalculé) Score SacreBLEU: {test_bleu_results_detailed['score']:.4f}")
            if 'precisions' in test_bleu_results_detailed:
                print(f"Précisions BLEU (1-4 grams): { [round(p, 4) for p in test_bleu_results_detailed['precisions']] }")
            print(f"Ratio de brièveté (BP): {test_bleu_results_detailed.get('bp', 'N/A'):.4f}")
            print(f"Longueur moyenne des prédictions: {np.mean([len(p.split()) for p in cleaned_preds]):.2f} mots")
            print(f"Longueur moyenne des références: {np.mean([len(l[0].split()) for l in cleaned_labels]):.2f} mots")

            # Ajout des détails au dictionnaire de métriques
            metrics["test_bleu_precisions"] = test_bleu_results_detailed.get('precisions')
            metrics["test_bp"] = test_bleu_results_detailed.get('bp')

            # Sauvegarde des prédictions/références
            output_prediction_file = os.path.join(run_name, "test_predictions_ru.txt")
            output_reference_file = os.path.join(run_name, "test_references_ru.txt")
            with open(output_prediction_file, "w", encoding="utf-8") as writer:
                writer.write("\n".join(cleaned_preds))
            with open(output_reference_file, "w", encoding="utf-8") as writer:
                writer.write("\n".join([ref[0] for ref in cleaned_labels]))
            print(f"Prédictions sauvegardées dans: {output_prediction_file}")
            print(f"Références sauvegardées dans: {output_reference_file}")

        except Exception as e:
            print(f"Erreur lors du recalcul détaillé de BLEU ou de la sauvegarde des fichiers: {e}")
            # Vérifier si c'est l'erreur d'espace disque
            if isinstance(e, OSError) and e.errno == 28:
                 print("ERREUR: Plus d'espace disque disponible pour sauvegarder les prédictions/références.")
            # Sauvegarder quand même les métriques principales si possible


    # Commentaire de log_metrics pour éviter TypeError potentiel
    # trainer.log_metrics("test", metrics)
    # Sauvegarder les métriques finales (tentative même si erreur disque avant)
    try:
        trainer.save_metrics("test", metrics)
        print("Métriques de test sauvegardées.")
    except Exception as save_e:
        print(f"Impossible de sauvegarder les métriques de test: {save_e}")
        if isinstance(save_e, OSError) and save_e.errno == 28:
             print("ERREUR: Plus d'espace disque disponible pour sauvegarder les métriques.")

except Exception as pred_e:
    print(f"Une erreur est survenue pendant la prédiction/évaluation : {pred_e}")


print("Évaluation finale terminée (ou tentée).")


# @title 7. Exemple d'Inférence (Traduction d'une phrase EN->RU)
# Essayer l'inférence même si l'évaluation a eu des soucis
try:
    print("\nExemple d'inférence avec le modèle NLLB fine-tuné...")
    sentence_en = "Machine translation is fascinating."
    print(f"Phrase source ({SOURCE_LANG_DATA}): {sentence_en}")

    # Tokenisation pour NLLB
    inputs = tokenizer(sentence_en, return_tensors="pt").to(device)

    # Spécifier la langue cible pour la génération avec NLLB
    forced_bos_token_id = tokenizer.lang_code_to_id[TARGET_LANG_NLLB]

    with torch.no_grad():
        outputs = model.generate(
            **inputs,
            forced_bos_token_id=forced_bos_token_id,
            max_length=MAX_TARGET_LENGTH,
            num_beams=4,
            early_stopping=True
        )

    translation_ru = tokenizer.decode(outputs[0], skip_special_tokens=True)
    print(f"Traduction ({TARGET_LANG_DATA}): {translation_ru}")

    # Autre exemple
    sentence_en_2 = "This model was fine-tuned on the full opus_books dataset."
    print(f"\nPhrase source ({SOURCE_LANG_DATA}): {sentence_en_2}")
    inputs_2 = tokenizer(sentence_en_2, return_tensors="pt").to(device)
    with torch.no_grad():
        outputs_2 = model.generate(
            **inputs_2,
            forced_bos_token_id=forced_bos_token_id,
            max_length=MAX_TARGET_LENGTH,
            num_beams=4,
            early_stopping=True
        )
    translation_ru_2 = tokenizer.decode(outputs_2[0], skip_special_tokens=True)
    print(f"Traduction ({TARGET_LANG_DATA}): {translation_ru_2}")

except Exception as inf_e:
    print(f"Une erreur est survenue pendant l'inférence: {inf_e}")


print(f"\nScript ({run_name}) terminé pour EN->RU (GPU).")

